In [1]:
import pyodbc
import pandas as pd
import math
import datetime
import numpy as np
# import timedelta
import pendulum
import matplotlib.pyplot as plt
import time
%matplotlib inline

start = time.time()

pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)
pd.set_option('display.width', 10000)

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                     'Server=222.112.162.221;'
                     'Database=moduintra;'
                     'UID=r114_analysis;'
                     'PWD=_r114!@#')

In [3]:
cursor = conn.cursor()

In [4]:
## 실거래가 가져오기
# 2018년 이후 실거래가, 매매, 서울, 강남구
# df_s = pd.read_sql_query("SELECT B.AP_CODE, A.BLDG_NAME, B.AP_KIND, B.AP_TYPE, A.CITY, A.SIGUNGU, A.BJDONG, A.YEAR, A.MONTH, A.CONTRACT_DATE, A.PRICE AS 실거래가 FROM LAND_RENT.dbo.MOCT_APT A INNER JOIN w93.modueum.eum.M_MOCT_MAST2 B ON A.BLDG_CD = B.apt_code AND A.BLDG_FLOOR_AREA = B.jun_area WHERE A.DEALING_GBN LIKE '매매' AND A.YEAR >= 2018", conn)
df_s = pd.read_sql_query("SELECT B.AP_CODE, A.BLDG_NAME, B.AP_KIND, B.AP_TYPE, A.CITY, A.SIGUNGU, A.BJDONG, A.YEAR, A.MONTH, A.CONTRACT_DATE, A.PRICE AS 실거래가 FROM LAND_RENT.dbo.MOCT_APT A INNER JOIN w93.modueum.eum.M_MOCT_MAST2 B ON A.BLDG_CD = B.apt_code AND A.BLDG_FLOOR_AREA = B.jun_area WHERE A.DEALING_GBN LIKE '매매' AND A.YEAR >= 2018 AND A.CITY LIKE '서울특별시'", conn)

## CONTRACT_DATE => 1114 조사일로 변환하는 작업
df_s["DAY"] = df_s["CONTRACT_DATE"].str.split('.').str[1]
df_s["YEAR"] = df_s["YEAR"].astype(int)
df_s["MONTH"] = df_s["MONTH"].astype(int)
df_s["DAY"] = df_s["DAY"].astype(int)
df_s = df_s.reset_index(drop=True)

### 실거래가일 변환 : 실거래일 다음으로 가까운 금요일 
### Ex ) 20200519 => 20200522
df_s["조사일"] = 0
for i in range(len(df_s)):
    df_s["조사일"][i] = pendulum.datetime(df_s["YEAR"][i], df_s["MONTH"][i], df_s["DAY"][i]).next(pendulum.FRIDAY).strftime('%Y%m%d')

df_s = df_s.rename(columns={'AP_CODE' : '아파트코드', 'AP_KIND' : '평형', "AP_TYPE" : '평형타입'})
df_s["아파트코드"] = df_s["아파트코드"].astype(str)
df_s["평형"] = df_s["평형"].astype(str)
df_s["평형타입"] = df_s["평형타입"].astype(str)
df_s = df_s[["아파트코드","평형","평형타입","실거래가","CONTRACT_DATE","조사일"]]

# [조사일] 컬럼 생성 : 114조사일기준 근접 실거래계약일 (매주 금)
df_s

c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,아파트코드,평형,평형타입,실거래가,CONTRACT_DATE,조사일
0,A01251312000008,00032,,54000,2.28,20200306
1,A01251312000001,00025,,46000,2.28,20200306
2,A01161331000033,00049,A,180000,11.8,20191115
3,A01161331000033,00049,B,180000,11.8,20191115
4,A01161331000033,00049,C,180000,11.8,20191115
...,...,...,...,...,...,...
1749053,A01231100100001,00027,,40500,9.15,20200918
1749054,A01231100100001,00027,,40000,9.19,20200925
1749055,A01231100800011,00032,G,110000,9.11,20200918
1749056,A03056071100002,00026,,17900,8.22,20200828


In [5]:
# 실거래가 2개이상 있는 경우 중복 기록
df_s["mg_id_s"] = df_s["아파트코드"] + df_s["평형"] + df_s["평형타입"] + df_s["조사일"].astype(str)
df_s["실거래_중복_num"] = df_s.groupby(df_s["mg_id_s"]).mg_id_s.transform('count')

# df_s["실거래_중복_num"] = df_s.groupby(['mg_id_s']).count().reset_index()['실거래가']
# df_s["실거래_중복_num"] = df_s.groupby(['아파트코드', '평형','평형타입','조사일']).count().reset_index()['실거래가']


df_s["실거래가"] = df_s["실거래가"].astype(int)
# df_s = df_s.drop('mg_id_s', axis = 1)
df_s
# df_ss = df_s[df_s.duplicated(["mg_id"], keep = False)].sort_values(["아파트코드","평형","평형타입"]).reset_index(drop=True)
# df_ss

,아파트코드,평형,평형타입,실거래가,CONTRACT_DATE,조사일,mg_id_s,실거래_중복_num
0,A01251312000008,00032,,54000,2.28,20200306,A012513120000080003220200306,1
1,A01251312000001,00025,,46000,2.28,20200306,A012513120000010002520200306,1
2,A01161331000033,00049,A,180000,11.8,20191115,A0116133100003300049A20191115,1
3,A01161331000033,00049,B,180000,11.8,20191115,A0116133100003300049B20191115,1
4,A01161331000033,00049,C,180000,11.8,20191115,A0116133100003300049C20191115,1
...,...,...,...,...,...,...,...,...
1749053,A01231100100001,00027,,40500,9.15,20200918,A012311001000010002720200918,1
1749054,A01231100100001,00027,,40000,9.19,20200925,A012311001000010002720200925,1
1749055,A01231100800011,00032,G,110000,9.11,20200918,A0123110080001100032G20200918,1
1749056,A03056071100002,00026,,17900,8.22,20200828,A030560711000020002620200828,1


In [6]:
len(df_s)

1749058

In [7]:
df_s[df_s["아파트코드"] == 'A01011350800102']

,아파트코드,평형,평형타입,실거래가,CONTRACT_DATE,조사일,mg_id_s,실거래_중복_num
973925,A01011350800102,00049,C,250000,6.8,20200612,A0101135080010200049C20200612,1
1405503,A01011350800102,00049,C,214000,8.24,20180831,A0101135080010200049C20180831,1


In [8]:
# # df_s.groupby(["mg_id"]).실거래가.transform('mean')
# df_s["sprice"] = df_s.groupby(["mg_id"])["실거래가"].transform('mean')
# df_s.sort_values(["mg_id","조사일"])

In [9]:
# def sprice_cal(x):
#     if (x["실거래_중복_num"] == 1):
#         return x["실거래가"]
#     elif (x["실거래_중복_num"] == 2):
#         return x.groupby("mg_id")["실거래가"].transform('mean')
# #     else :
# #         return x["실거래가"]

In [10]:
# df_s = df_s.assign(sprice = df_s.apply(sprice_cal, axis = 1))
# df_s

In [11]:
## 114 과거시세 가져오기
## 2018년 이후 , 서울, 강남구
# df_114 = pd.read_sql_query("SELECT * FROM (SELECT D.아파트코드, D.아파트명, D.입주연도,D.도시, D.구시군 , D.읍면동, C.평형, C.평형타입,C.세대수, C.주택형, C.분양가, C.조사일, C.매매하한가, C.매매상한가, C.전세하한가, C.전세상한가, C.최근변경일 FROM (SELECT A.아파트코드, A.평형, A.평형타입, A.세대수, B.조사일, A.주택형, A.분양가, B.매매하한가, B.매매상한가, B.전세하한가, B.전세상한가, B.최근변경일 FROM modu.P_APTSUB AS A INNER JOIN modu.P_APTHPRICE AS B ON A.아파트코드 = B.아파트코드 AND A.평형 = B.평형 AND A.평형타입 = B. 평형타입) AS C INNER JOIN modu.P_APT AS D ON C.아파트코드 = D.아파트코드) AS E WHERE E.매매하한가 > 0 AND E.조사일 NOT LIKE '% %' AND E.조사일 > 20180000 AND E.최근변경일 IS NOT NULL", conn)
df_114 = pd.read_sql_query("SELECT * FROM (SELECT D.아파트코드, D.아파트명, D.입주연도,D.도시, D.구시군 , D.읍면동, C.평형, C.평형타입,C.세대수, C.주택형, C.분양가, C.조사일, C.매매하한가, C.매매상한가, C.전세하한가, C.전세상한가, C.최근변경일 FROM (SELECT A.아파트코드, A.평형, A.평형타입, A.세대수, B.조사일, A.주택형, A.분양가, B.매매하한가, B.매매상한가, B.전세하한가, B.전세상한가, B.최근변경일 FROM modu.P_APTSUB AS A INNER JOIN modu.P_APTHPRICE AS B ON A.아파트코드 = B.아파트코드 AND A.평형 = B.평형 AND A.평형타입 = B. 평형타입) AS C INNER JOIN modu.P_APT AS D ON C.아파트코드 = D.아파트코드) AS E WHERE E.도시 = '서울특별시' AND E.매매하한가 > 0 AND E.조사일 NOT LIKE '% %' AND E.조사일 > 20180000", conn)

df_114["aptprice"] = df_114[['매매하한가','매매상한가']].mean(axis=1)
df_114["aptprice_all_sum"] = df_114["aptprice"] * df_114["세대수"]
df_114["조사일"] = df_114["조사일"].astype(int)
df_114["아파트코드"] = df_114["아파트코드"].astype(str)
df_114["평형"] = df_114["평형"].astype(str)
df_114["평형타입"] = df_114["평형타입"].astype(str)

## 전체 식별코드 생성
# df_114["mg_id"] = df_114["아파트코드"] + df_114["평형"] + df_114["평형타입"] + pd.to_datetime(df_114["조사일"], format="%Y%m%d").astype(str).str.replace('-','')
df_114["mg_id"] = df_114["아파트코드"] + df_114["평형"] + df_114["평형타입"]

df_114 = df_114[-df_114["입주연도"].astype(str).str.startswith('0')]
df_114.head()

,아파트코드,아파트명,입주연도,도시,구시군,읍면동,평형,평형타입,세대수,주택형,분양가,조사일,매매하한가,매매상한가,전세하한가,전세상한가,최근변경일,aptprice,aptprice_all_sum,mg_id
0,A02164800900021,송산주공2단지,20020617,경기도,의정부시,민락동,00019,B,33,64.8800,0,20201023,15000,17500,13500,15000,20201009,16250.0,536250.0,A0216480090002100019B
1,A02164800900033,민락2반도유보라아이비파크(B10),20170926,경기도,의정부시,낙양동,00034,B,352,111.6385,29560,20201023,38000,40000,27000,31000,20201017,39000.0,13728000.0,A0216480090003300034B
2,A02164800900035,호반베르디움3차,20171013,경기도,의정부시,민락동,00034,A,384,112.0730,30680,20201023,31680,33180,0,0,20201016,32430.0,12453120.0,A0216480090003500034A
3,A02164801000002,금용,199606,경기도,의정부시,가능동,00032,A,17,105.7856,0,20201023,22000,25000,9000,10000,20201016,23500.0,399500.0,A0216480100000200032A
4,A02164801000006,신도,199408,경기도,의정부시,가능동,00031,,60,103.1500,0,20201023,18000,19000,13000,14000,20190524,18500.0,1110000.0,A0216480100000600031


In [12]:
len(df_114) # 153492

9361410

In [13]:
# 변경전 시가총액 변동율
dat1 = df_114.groupby(["조사일","구시군"])["aptprice_all_sum"].sum()
dat1 = dat1.reset_index()
dat1["change(%)"] = (dat1["aptprice_all_sum"].diff() / dat1["aptprice_all_sum"]) * 100
dat1.tail()

# 변경후 시가총액 변동율
#dat2

,조사일,구시군,aptprice_all_sum,change(%)
26926,20201023,홍성군,2.276018e+08,-2143.417297
26927,20201023,홍천군,5.509290e+07,-313.123651
26928,20201023,화성시,6.351840e+09,99.132647
26929,20201023,화순군,5.172400e+07,-12180.256967
26930,20201023,횡성군,2.210440e+07,-133.998661


In [14]:
df_114_s = df_114[["아파트코드", "아파트명","입주연도","도시","구시군","읍면동","평형","평형타입","mg_id","세대수","조사일","aptprice","최근변경일"]]

In [15]:
len(df_114_s)

9361410

In [16]:
# 오늘날짜 
# dt = datetime.date.today()
dt = pendulum.now()

# 오늘 날짜로부터 8주 이전 : 실거래가 신고일 + 114 DB 이관적용일 고려
bf_6_res = str(dt.subtract(weeks=8)).split('T')[0].replace("-",'')
bf_6_res = pendulum.datetime(int(bf_6_res[:4]), int(bf_6_res[4:6]), int(bf_6_res[6:8])).next(pendulum.FRIDAY).strftime('%Y%m%d')
bf_6_res

'20200828'

In [17]:
# 최근변경일이 8주 이전인 114 시세 필터
# df_114_s["최근변경일"] = df_114_s["최근변경일"].astype(int)
# df_114_s = df_114_s[df_114_s["최근변경일"] < int(bf_6_res)]

# 최근데이터 8주 이내 필터
# df_114_s = df_114_s[df_114_s["최근변경일"] >= int(bf_6_res)]
len(df_114_s)

9361410

In [18]:
r114_mg = pd.merge(df_114_s, df_s, how = 'left', on=["아파트코드","평형", "평형타입","조사일"])
r114_mg = r114_mg.rename(columns={'aptprice' : '114매매평균가'})
# r114_mg[-r114_mg["실거래가"].isnull()]
# r114_mg_sg = r114_mg[(-r114_mg["실거래가"].isnull())]
# r114_mg["mg_id"] = r114_mg["아파트코드"] + r114_mg["평형"] + r114_mg["평형타입"] + r114_mg["최근변경일"]
r114_mg

,아파트코드,아파트명,입주연도,도시,구시군,읍면동,평형,평형타입,mg_id,세대수,조사일,114매매평균가,최근변경일,실거래가,CONTRACT_DATE,mg_id_s,실거래_중복_num
0,A02164800900021,송산주공2단지,20020617,경기도,의정부시,민락동,00019,B,A0216480090002100019B,33,20201023,16250.0,20201009,NaN,NaN,NaN,NaN
1,A02164800900033,민락2반도유보라아이비파크(B10),20170926,경기도,의정부시,낙양동,00034,B,A0216480090003300034B,352,20201023,39000.0,20201017,NaN,NaN,NaN,NaN
2,A02164800900035,호반베르디움3차,20171013,경기도,의정부시,민락동,00034,A,A0216480090003500034A,384,20201023,32430.0,20201016,NaN,NaN,NaN,NaN
3,A02164801000002,금용,199606,경기도,의정부시,가능동,00032,A,A0216480100000200032A,17,20201023,23500.0,20201016,NaN,NaN,NaN,NaN
4,A02164801000006,신도,199408,경기도,의정부시,가능동,00031,,A0216480100000600031,60,20201023,18500.0,20190524,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9738419,A08036812700005,남외2차푸르지오,20070731,울산광역시,중구,남외동,00034,,A0803681270000500034,236,20180105,39750.0,20180802,NaN,NaN,NaN,NaN
9738420,A08036813100002,유곡e편한세상,20100301,울산광역시,중구,유곡동,00048,B,A0803681310000200048B,168,20180105,50500.0,20160603,NaN,NaN,NaN,NaN
9738421,A08036813200001,진영동부,199402,울산광역시,중구,태화동,00049,,A0803681320000100049,70,20180105,28500.0,20150821,NaN,NaN,NaN,NaN
9738422,A08036813200004,삼익세라믹1차,199012,울산광역시,중구,태화동,00031,,A0803681320000400031,96,20180105,18250.0,20160909,NaN,NaN,NaN,NaN


In [19]:
len(r114_mg)

9738424

In [20]:
# df_114_int = pd.concat([dd_ano, dd_all], axis = 0).sort_values(["아파트코드","평형","평형타입","조사일","최근변경일"])
# df_114_int["aptprice_all_sum"] = df_114_int["114매매평균가"] * df_114_int["세대수"]
# df_114_int["조사일"] = df_114_int["조사일"].astype(int)
# df_114_int = df_114_int[-df_114_int["입주연도"].astype(str).str.startswith('0')]
# df_114_int

In [21]:
# 변경전 시가총액 변동율
# dat1 = df_114.groupby(["조사일","구시군"])["aptprice_all_sum"].sum()
# dat1 = dat1.reset_index()
# dat1["change(%)"] = (dat1["aptprice_all_sum"].diff() / dat1["aptprice_all_sum"]) * 100
# dat1.tail()

# # 변경후 시가총액 변동율
# dat2 = df_114_int.groupby(["조사일","구시군"])["aptprice_all_sum"].sum()
# dat2 = dat2.reset_index()
# dat2["change(%)"] = (dat2["aptprice_all_sum"].diff() / dat2["aptprice_all_sum"]) * 100
# dat2.tail()

In [22]:
## 감정원 시세 가져오기
# df_kab = pd.read_sql_query("SELECT r114_기본정보코드, r114_평형, r114_평형타입, 매매하한가, 매매상한가 , 임대하한가, 임대상한가, 조사일 FROM modu.한국감정원시세누적정보 WHERE 조사일 > 20180100", conn)

# df_kab = pd.read_sql_query("SELECT A.r114_기본정보코드, A.r114_평형, A.r114_평형타입, B.구시군, A.매매하한가, A.매매상한가 , A.임대하한가, A.임대상한가, A.조사일 FROM modu.한국감정원시세누적정보 AS A INNER JOIN [modu].[한국감정원_R114_기본정보매핑] AS B ON A.r114_기본정보코드 = B.r114_기본정보코드 WHERE A.조사일 > 20180100", conn)
df_kab = pd.read_sql_query("SELECT A.r114_기본정보코드, A.r114_평형, A.r114_평형타입, B.도시, B.구시군, A.매매하한가, A.매매상한가 , A.임대하한가, A.임대상한가, A.조사일 FROM modu.한국감정원시세누적정보 AS A INNER JOIN [modu].[한국감정원_R114_기본정보매핑] AS B ON A.r114_기본정보코드 = B.r114_기본정보코드 WHERE A.조사일 > 20180100 AND B.도시 LIKE '서울특별시'", conn)

df_kab["감정원매매평균"] = df_kab[['매매하한가','매매상한가']].mean(axis=1)
df_kab = df_kab.rename(columns={'r114_기본정보코드' : '아파트코드', 'r114_평형' : '평형', 'r114_평형타입' : '평형타입'})
df_kab = df_kab[["아파트코드", "평형", "평형타입", "구시군","조사일", "감정원매매평균","매매상한가", "매매하한가"]]

df_kab["YEAR"] = df_kab["조사일"].str[:4].astype(int)
df_kab["MONTH"] = df_kab["조사일"].str[4:6].astype(int)
df_kab["DAY"] = df_kab["조사일"].str[6:8].astype(int)

df_kab["조사일_수정"] = 0
for i in range(len(df_kab)):
    df_kab["조사일_수정"][i] = pendulum.datetime(df_kab["YEAR"][i], df_kab["MONTH"][i], df_kab["DAY"][i]).previous(pendulum.FRIDAY).strftime('%Y%m%d')

# df_kab.info()

c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [23]:
len(df_kab)

36880597

In [24]:
df_kab["조사일_수정"] = df_kab["조사일_수정"].astype(int)
df_kab["아파트코드"] = df_kab["아파트코드"].astype(str)
df_kab["평형"] = df_kab["평형"].astype(str)
df_kab["평형타입"] = df_kab["평형타입"].astype(str)
df_kab = df_kab[["아파트코드", "평형", "평형타입", "구시군", "조사일_수정", "감정원매매평균","매매상한가", "매매하한가"]]
df_kab = df_kab.rename(columns={'조사일_수정' : '조사일', "매매상한가" : "감정원_상한가", "매매하한가" : '감정원_하한가'})
df_kab = df_kab.drop_duplicates(["아파트코드", "평형","평형타입","조사일"], keep = 'first')
df_kab["감정원_가격폭"] = df_kab["감정원_상한가"] - df_kab["감정원_하한가"]
df_kab

,아파트코드,평형,평형타입,구시군,조사일,감정원매매평균,감정원_상한가,감정원_하한가,감정원_가격폭
0,A02014122100002,00025,,고양시,20181214,29000.0,32000,26000,6000
2,A02014122100002,00032,,고양시,20181214,38500.0,40000,37000,3000
4,A03096040300001,00037,,사하구,20181214,16400.0,17800,15000,2800
9,A03096040300001,00045,,사하구,20181214,20750.0,22200,19300,2900
14,A03096040300001,00050,A,사하구,20181214,21450.0,23200,19700,3500
...,...,...,...,...,...,...,...,...,...
36880584,A02114294500006,00031,,시흥시,20190104,21000.0,22000,20000,2000
36880587,A02114294500006,00033,,시흥시,20190104,22500.0,24000,21000,3000
36880590,A02114294500006,00049,,시흥시,20190104,31000.0,33000,29000,4000
36880593,A02114294500008,00022,,시흥시,20190104,17000.0,18000,16000,2000


In [25]:
df_kab["감정원_가격폭"] = df_kab["감정원_상한가"] - df_kab["감정원_하한가"]

In [44]:
df_114_s_kab = pd.merge(r114_mg, df_kab, on=["아파트코드", "평형","평형타입","조사일"], how = 'inner') # 
df_114_s_kab["최근변경일"] = df_114_s_kab["최근변경일"].dropna()
df_114_s_kab = df_114_s_kab[-(df_114_s_kab["최근변경일"] == '')]
df_114_s_kab["최근변경일"] = df_114_s_kab["최근변경일"].astype(int)
df_114_s_kab

,아파트코드,아파트명,입주연도,도시,구시군_x,읍면동,평형,평형타입,mg_id,세대수,조사일,114매매평균가,최근변경일,실거래가,CONTRACT_DATE,mg_id_s,실거래_중복_num,구시군_y,감정원매매평균,감정원_상한가,감정원_하한가,감정원_가격폭
0,A01021340500010,광나루삼성,200103,서울특별시,강동구,암사동,00023,B,A0102134050001000023B,112,20201009,82500.0,20201007,NaN,NaN,NaN,NaN,강동구,80000.0,85000,75000,10000
1,A01021340500029,포스파크,20020902,서울특별시,강동구,암사동,00042,,A0102134050002900042,19,20201009,55000.0,20201009,NaN,NaN,NaN,NaN,강동구,92500.0,98000,87000,11000
2,A01021340500037,강동현대홈타운,20040825,서울특별시,강동구,암사동,00033,,A0102134050003700033,346,20201009,106500.0,20201005,NaN,NaN,NaN,NaN,강동구,107500.0,112000,103000,9000
3,A01021340500054,롯데캐슬퍼스트,20080926,서울특별시,강동구,암사동,00034,A,A0102134050005400034A,132,20201009,129000.0,20201007,NaN,NaN,NaN,NaN,강동구,126500.0,132000,121000,11000
4,A01021340600003,중앙하이츠,199512,서울특별시,강동구,둔촌동,00033,,A0102134060000300033,62,20201009,84000.0,20200925,NaN,NaN,NaN,NaN,강동구,85000.0,88000,82000,6000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6324868,A08036812500017,에일린의뜰3차,20140730,울산광역시,중구,유곡동,00033,A,A0803681250001700033A,55,20180105,43500.0,20160603,NaN,NaN,NaN,NaN,중구,42500.0,44000,41000,3000
6324869,A08036812500017,에일린의뜰3차,20140730,울산광역시,중구,유곡동,00038,A,A0803681250001700038A,328,20180105,47000.0,20160603,NaN,NaN,NaN,NaN,중구,46500.0,48000,45000,3000
6324870,A08036812700005,남외2차푸르지오,20070731,울산광역시,중구,남외동,00034,,A0803681270000500034,236,20180105,39750.0,20180802,NaN,NaN,NaN,NaN,중구,40000.0,41500,38500,3000
6324871,A08036813100002,유곡e편한세상,20100301,울산광역시,중구,유곡동,00048,B,A0803681310000200048B,168,20180105,50500.0,20160603,NaN,NaN,NaN,NaN,중구,49500.0,52500,46500,6000


In [45]:
# df_114_s_kab.groupby('최근변경일').count()["아파트코드"]
len(df_114_s_kab) # 6324873

6324818

In [48]:
# 실거래 이상여부 정의 (2조건이 OR일떄)
## 조건1 : (감정원 상한가 - 실거래가)의 절대값이 감정원 상한가의 10% 보다 큰 경우 이거나
## 조건2 : (실거래가 - 감정원 하한가)의 절대값이 감정원 하한가의 10% 보다 큰 경우

# df_114_s_kab["실거래이상치1"] = abs(df_114_s_kab["실거래가"] - df_114_s_kab["감정원_상한가"]) > df_114_s_kab["감정원_상한가"] * 0.1
# df_114_s_kab["실거래이상치2"] = abs(df_114_s_kab["실거래가"] - df_114_s_kab["감정원_하한가"]) > df_114_s_kab["감정원_하한가"] * 0.1

# 상/하한 이상치 변동폭 수정
df_114_s_kab["실거래이상치1"] = df_114_s_kab["실거래가"] > df_114_s_kab["감정원_상한가"] + (df_114_s_kab["감정원_상한가"] * 0.4)
df_114_s_kab["실거래이상치2"] = df_114_s_kab["실거래가"] < df_114_s_kab["감정원_하한가"] - (df_114_s_kab["감정원_하한가"] * 0.1)

In [49]:
len(df_114_s_kab)

6324818

In [50]:
# df_114_s_kab.loc[(df_114_s_kab["실거래이상치"] == True) & (df_114_s_kab["실거래가"].isnull() == False), '114매매평균가'] = df_114_s_kab[(df_114_s_kab["실거래이상치"] == True) & (df_114_s_kab["실거래가"].isnull() == False)].groupby(['아파트코드', '평형','평형타입', '조사일'])["실거래가"].mean().reset_index()["실거래가"]
# df_114_s_kab.loc[(df_114_s_kab["실거래가"].isnull() == False), '114매매평균가'] = df_114_s_kab[(df_114_s_kab["실거래가"].isnull() == False)].groupby(['mg_id'])["실거래가"].mean().reset_index()["실거래가"]

In [51]:
# 실거래가 평균값 (보간법 적용시작점) 위한 준비
# df_114_s_kab_1 : 실거래가 있는경우
df_114_s_kab_1 = df_114_s_kab[df_114_s_kab["실거래가"].isnull() == False]

# df_114_s_kab_2 : 실거래가 없는경우
df_114_s_kab_2 = df_114_s_kab[df_114_s_kab["실거래가"].isnull() == True]
df_114_s_kab_2["실거래가_평균"] = np.NaN

c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [52]:
len(df_114_s_kab_1)

1121645

In [53]:
len(df_114_s_kab_2)

5203173

In [54]:
# df_114_s_kab_1_1 :  실거래가 이상치 기준에 해당 없는 실거래가
df_114_s_kab_1_1 = df_114_s_kab_1[(df_114_s_kab_1["실거래이상치1"] == False) & (df_114_s_kab_1["실거래이상치2"] == False)]

# df_114_s_kab_1_1["mg_id_s"] = df_114_s_kab_1_1["아파트코드"] + df_114_s_kab_1_1["평형"] + df_114_s_kab_1_1["평형타입"] + df_114_s_kab_1_1["조사일"].astype(str)

df_114_s_kab_1_1 = df_114_s_kab_1_1.groupby(["아파트코드", "평형","평형타입","조사일"])['실거래가'].mean().reset_index()
# df_114_s_kab_1_1 = df_114_s_kab_1_1.groupby(["mg_id_s"])['실거래가'].mean().reset_index()

df_114_s_kab_1_1 = df_114_s_kab_1_1.rename(columns={'실거래가' : '실거래가_평균'})
df_114_s_kab_1_1["실거래가_평균"] = round(df_114_s_kab_1_1["실거래가_평균"], -2)

df_114_s_kab_1_1.head()

,아파트코드,평형,평형타입,조사일,실거래가_평균
0,A01011350100001,00033,,20180316,135000.0
1,A01011350100001,00033,,20180810,128500.0
2,A01011350100001,00033,,20180817,135000.0
3,A01011350100001,00033,,20180824,135000.0
4,A01011350100001,00033,,20180914,147000.0


In [55]:
len(df_114_s_kab_1_1)

810060

In [56]:
df_114_s_kab_1_1_1 = pd.merge(df_114_s_kab_1, df_114_s_kab_1_1, on=["아파트코드","평형","평형타입","조사일"],how = 'inner')
# df_114_s_kab_1_1_1["114매매평균가"] = round(df_114_s_kab_1_1_1["실거래가_평균"],-3)
# df_114_s_kab_1_1_1 = df_114_s_kab_1_1_1.drop(['실거래가_평균'], axis=1)
df_114_s_kab_1_1_1

,아파트코드,아파트명,입주연도,도시,구시군_x,읍면동,평형,평형타입,mg_id,세대수,조사일,114매매평균가,최근변경일,실거래가,CONTRACT_DATE,mg_id_s,실거래_중복_num,구시군_y,감정원매매평균,감정원_상한가,감정원_하한가,감정원_가격폭,실거래이상치1,실거래이상치2,실거래가_평균
0,A01031421000009,SK북한산시티,200111,서울특별시,강북구,미아동,00024,B,A0103142100000900024B,18,20201009,60000.0,20201007,60000.0,10.7,A0103142100000900024B20201009,2.0,강북구,59500.0,65000,54000,11000,False,False,62400.0
1,A01031421000009,SK북한산시티,200111,서울특별시,강북구,미아동,00024,B,A0103142100000900024B,18,20201009,60000.0,20201007,64800.0,10.8,A0103142100000900024B20201009,2.0,강북구,59500.0,65000,54000,11000,False,False,62400.0
2,A01031421000031,삼각산아이원,20031230,서울특별시,강북구,미아동,00031,,A0103142100003100031,805,20201009,70000.0,20201005,70000.0,10.5,A010314210000310003120201009,2.0,강북구,70000.0,74000,66000,8000,False,False,70000.0
3,A01031421000031,삼각산아이원,20031230,서울특별시,강북구,미아동,00031,,A0103142100003100031,805,20201009,70000.0,20201005,70000.0,10.8,A010314210000310003120201009,2.0,강북구,70000.0,74000,66000,8000,False,False,70000.0
4,A01031421000009,SK북한산시티,200111,서울특별시,강북구,미아동,00024,A,A0103142100000900024A,1177,20201009,60000.0,20201007,60000.0,10.7,A0103142100000900024A20201009,2.0,강북구,59500.0,65000,54000,11000,False,False,62400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104255,A08016800100116,문수로2차아이파크 2단지,20131231,울산광역시,남구,신정동,00044,B,A0801680010011600044B,50,20180105,72000.0,20200102,73000.0,1.4,A0801680010011600044B20180105,1.0,남구,71500.0,72000,71000,1000,False,False,73000.0
1104256,A08016800300004,주공2단지,199312,울산광역시,남구,달동,00015,,A0801680030000400015,480,20180105,8750.0,20121116,11900.0,1.2,A080168003000040001520180105,1.0,남구,12750.0,13600,11900,1700,False,False,11900.0
1104257,A08036812500016,에일린의뜰2차,20131031,울산광역시,중구,장현동,00029,B,A0803681250001600029B,72,20180105,33500.0,20160603,29200.0,1.4,A0803681250001600029B20180105,1.0,중구,33000.0,34500,31500,3000,False,False,29200.0
1104258,A08036813100003,푸르지오,20100430,울산광역시,중구,유곡동,00032,,A0803681310000300032,236,20180105,43000.0,20160603,41000.0,1.3,A080368131000030003220180105,1.0,중구,40500.0,42500,38500,4000,False,False,41000.0


In [57]:
len(df_114_s_kab_1_1_1)

1104260

In [58]:
df_114_s_kab_3 = pd.concat([df_114_s_kab_1_1_1,df_114_s_kab_2 ], axis=0).reset_index(drop=True).sort_values(["아파트코드","평형","평형타입","조사일"])
df_114_s_kab_3

,아파트코드,아파트명,입주연도,도시,구시군_x,읍면동,평형,평형타입,mg_id,세대수,조사일,114매매평균가,최근변경일,실거래가,CONTRACT_DATE,mg_id_s,실거래_중복_num,구시군_y,감정원매매평균,감정원_상한가,감정원_하한가,감정원_가격폭,실거래이상치1,실거래이상치2,실거래가_평균
6256429,A01011350100001,동현,198609,서울특별시,강남구,논현동,00033,,A0101135010000100033,380,20180105,98000.0,20170706,NaN,NaN,NaN,NaN,강남구,107500.0,110000,105000,5000,False,False,NaN
6221078,A01011350100001,동현,198609,서울특별시,강남구,논현동,00033,,A0101135010000100033,380,20180112,98000.0,20170706,NaN,NaN,NaN,NaN,강남구,107500.0,110000,105000,5000,False,False,NaN
6177155,A01011350100001,동현,198609,서울특별시,강남구,논현동,00033,,A0101135010000100033,380,20180119,98000.0,20170706,NaN,NaN,NaN,NaN,강남구,107500.0,110000,105000,5000,False,False,NaN
6152657,A01011350100001,동현,198609,서울특별시,강남구,논현동,00033,,A0101135010000100033,380,20180126,98000.0,20170706,NaN,NaN,NaN,NaN,강남구,107500.0,110000,105000,5000,False,False,NaN
6151191,A01011350100001,동현,198609,서울특별시,강남구,논현동,00033,,A0101135010000100033,380,20180202,98000.0,20170706,NaN,NaN,NaN,NaN,강남구,113000.0,116000,110000,6000,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299275,A99154652400001,위례그린파크푸르지오,20160130,경기도,하남시,학암동,00039,D,A9915465240000100039D,106,20200904,96000.0,20200904,NaN,NaN,NaN,NaN,하남시,129000.0,138000,120000,18000,False,False,NaN
1257416,A99154652400001,위례그린파크푸르지오,20160130,경기도,하남시,학암동,00039,D,A9915465240000100039D,106,20200911,96000.0,20200911,NaN,NaN,NaN,NaN,하남시,129000.0,138000,120000,18000,False,False,NaN
1232792,A99154652400001,위례그린파크푸르지오,20160130,경기도,하남시,학암동,00039,D,A9915465240000100039D,106,20200918,96000.0,20200911,NaN,NaN,NaN,NaN,하남시,129000.0,138000,120000,18000,False,False,NaN
1178374,A99154652400001,위례그린파크푸르지오,20160130,경기도,하남시,학암동,00039,D,A9915465240000100039D,106,20200925,96000.0,20200925,NaN,NaN,NaN,NaN,하남시,129000.0,138000,120000,18000,False,False,NaN


In [59]:
len(df_114_s_kab_3)

6307433

In [60]:
# 최근변경일 ~ 
# df_114_s_kab_3 = df_114_s_kab_3[df_114_s_kab_3["최근변경일"] < int(bf_6_res)]

In [61]:
len(df_114_s_kab_3)

6307433

In [62]:
len(df_114_s_kab)

6324818

In [63]:
r114_mg_recent = df_114_s_kab_3.groupby(["아파트코드","평형","평형타입","조사일"])["최근변경일"].max().sort_values().reset_index()

# r114_mg_recent["mg_id"] = r114_mg_recent["아파트코드"] + r114_mg_recent["평형"] + r114_mg_recent["평형타입"] + pd.to_datetime(r114_mg_recent["조사일"], format="%Y%m%d").astype(str).str.replace('-','')
r114_mg_recent["mg_id"] = r114_mg_recent["아파트코드"] + r114_mg_recent["평형"] + r114_mg_recent["평형타입"]

# r114_mg_recent["mg_id"]
# r114_mg_recent["mg_id_s"] = r114_mg_recent["mg_id"].str[:-8]
# r114_mg_recent["mg_dt"] = r114_mg_recent["mg_id"].str[-8:]
# r114_mg_recent = r114_mg_recent[["mg_id", "mg_dt"]]

r114_mg_recent

,아파트코드,평형,평형타입,조사일,최근변경일,mg_id
0,A10096414800001,00022,,20180105,20090612,A1009641480000100022
1,A10096414800001,00020,,20180406,20090612,A1009641480000100020
2,A10096414800001,00022,,20180406,20090612,A1009641480000100022
3,A10096414800001,00022,,20180323,20090612,A1009641480000100022
4,A10096414800001,00022,,20180316,20090612,A1009641480000100022
...,...,...,...,...,...,...
6013228,A02524113600015,00022,A,20201009,20201021,A0252411360001500022A
6013229,A02524113600015,00022,A,20200925,20201021,A0252411360001500022A
6013230,A02524113600015,00022,A,20200918,20201021,A0252411360001500022A
6013231,A02524113600015,00022,A,20200320,20201021,A0252411360001500022A


In [64]:
# df_114_s_kab_3.index = pd.to_datetime(df_114_s_kab_3["조사일"], format='%Y%m%d')
# df_114_s_kab_3.index.name = None
# df_114_s_kab_3.head()

In [ ]:
dd_all = pd.DataFrame(columns=df_114_s_kab_3.columns.values)

for i in pd.unique(r114_mg_recent["mg_id"]):
# for i in pd.unique(df_s["mg_id_s"])[:3]:

    # 보간법 적용
    dd = df_114_s_kab_3[(df_114_s_kab_3["mg_id"] == i)].sort_values(by=["조사일"], ascending = True).reset_index()
    
    dd["114매매평균가"][0] = dd["114매매평균가"][0]
#     dd["114매매평균가"][0] = dd["감정원매매평균"][0]
#     ## 1.기존값 -> NULL 처리
    dd["114매매평균가"][1:] = np.NaN
    
    ## 2. NULL중 실거래가 있는 해당주는 기존114매매평균가 -> 실거래가로 대체
    dd["114매매평균가"][dd[-dd["실거래가_평균"].isnull()].index] = round(dd["실거래가_평균"][dd[-dd["실거래가_평균"].isnull()].index], -2)
    
    dd.index = pd.to_datetime(dd["조사일"], format='%Y%m%d')
    dd.index.name = None
    
#     dd["114매매평균가_수정"] = dd["114매매평균가"]
    
    ## 3. NULL처리된 부분 interpolate() 적용
#     dd["114매매평균가"] = round(dd["114매매평균가"].interpolate(),-2)
    dd["114매매평균가"] = round(dd["114매매평균가"].interpolate(method = 'time'),-3)

    dd_all = pd.concat([dd_all,dd], axis = 0)
#     print(dd)
    del dd

c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel i

In [ ]:
dd_all = dd_all.drop(['실거래이상치1', '실거래이상치2', '실거래가_평균'], axis = 1)
dd_all

In [ ]:
len(dd_all)

In [ ]:
# res_all = pd.concat([dd_all, df_114_s_kab_ano], axis = 0).sort_values(["아파트코드","평형","평형타입","조사일"])

# 상/하한 기준 : 감정원_상한가 - 감정원_하한가
# res_all["감정원_가격폭"] = res_all["감정원_상한가"] - res_all["감정원_하한가"]
# res_all["114_매매하한가추정"] = res_all["114매매평균가"] - res_all["감정원_가격폭"]
dd_all = dd_all.sort_values(["아파트코드","평형","평형타입","조사일"])
dd_all

In [ ]:
len(dd_all)

In [ ]:
# 변경전 시가총액 변동율
dat1 = df_114.groupby(["조사일","구시군"])["aptprice_all_sum"].sum()
dat1 = dat1.reset_index()
dat1["change(%)"] = (dat1["aptprice_all_sum"].diff() / dat1["aptprice_all_sum"]) * 100
dat1.tail()

In [ ]:
dd_all_j = dd_all[["아파트코드", "평형","평형타입","조사일","114매매평균가", "실거래가","실거래_중복_num","감정원_가격폭","감정원매매평균"]]
dd_all_j

In [ ]:
dd_all_j

In [ ]:
# int_res = pd.merge(df_114, dd_all_j, on = ["아파트코드","평형","평형타입","조사일"], how = 'right')
int_res = dd_all_j
int_res

In [ ]:
int_res["114_보정후_하한가"] = int_res["114매매평균가"] - (int_res["감정원_가격폭"] / 2)
int_res["114_보정후_상한가"] = int_res["114매매평균가"] + (int_res["감정원_가격폭"] / 2)
int_res["114평균"] =  int_res[['114_보정후_하한가','114_보정후_상한가']].mean(axis=1)

In [ ]:
int_res

In [ ]:
int_res["114평균"] =  int_res[['114_보정후_하한가','114_보정후_상한가']].mean(axis=1)
# int_res["114평균"][int_res[int_res["114매매평균가"].isnull() == False].index] = int_res["114매매평균가"][int_res[int_res["114매매평균가"].isnull() == False].index]
# int_res = int_res.drop(columns=["aptprice_all_sum"])
# int_res["aptprice_all_sum"] = int_res["114평균"] * int_res["세대수"]
# int_res[int_res["114매매평균가"].isnull() == False]
len(int_res)

In [ ]:
int_res.head()

In [ ]:
int_res = int_res.sort_values(['아파트코드','평형','평형타입','조사일','114평균'], ascending = False)
# int_res = int_res.drop_duplicates(['아파트코드','평형','평형타입','조사일'],keep='first')
len(int_res)

In [ ]:
# # 변경후 시가총액 변동율
# dat2 = int_res.groupby(["조사일","구시군"])["aptprice_all_sum"].sum()
# dat2 = dat2.reset_index()
# dat2["change(%)"] = (dat2["aptprice_all_sum"].diff() / dat2["aptprice_all_sum"]) * 100
# dat2.tail()

In [ ]:
len(df_114)

In [ ]:
int_res[(int_res["아파트코드"] == "A01011350900003") & (int_res["평형"] == "00024")][["아파트코드","조사일", "114평균"]].sort_values(by=["조사일"])

In [ ]:
int_res[(int_res["아파트코드"] == "A01011350900003") & (int_res["평형"] == "00024")].sort_values(by=["조사일"])

In [ ]:
int_res[(int_res["아파트코드"] == "A01011350900003") & (int_res["평형"] == "00024")][["아파트코드","조사일", "실거래가", "실거래_중복_num","114평균"]].sort_values(by=["조사일"])

In [ ]:
len(pd.unique(int_res["아파트코드"]))

In [ ]:
pd.unique(int_res["실거래_중복_num"])

In [ ]:
int_res[["아파트코드","실거래_중복_num"]].groupby(by=["실거래_중복_num"]).count()

In [ ]:
int_res[int_res["실거래_중복_num"] == 6][["아파트코드","평형","조사일", "실거래가", "실거래_중복_num","114평균"]]

In [ ]:
int_res[(int_res["아파트코드"] == "A01011350100003") & (int_res["평형"] == "00016")][["아파트코드","조사일", "실거래가", "실거래_중복_num","114평균"]].sort_values(by=["조사일"])

In [ ]:
int_res[(int_res["아파트코드"] == "A01011350800099") & (int_res["평형"] == "00024") & (int_res["평형타입"] == 'A')][["아파트코드","평형","평형타입","조사일", "실거래가", "실거래_중복_num","114평균"]].sort_values(by=["조사일"])

In [ ]:
int_res[(int_res["아파트코드"] == "A01011350800099") & (int_res["평형"] == "00024") & (int_res["평형타입"] == 'B')][["아파트코드","평형","평형타입","조사일", "실거래가", "실거래_중복_num","114평균"]].sort_values(by=["조사일"])

In [ ]:
int_res[(int_res["아파트코드"] == "A01011350800099") & (int_res["평형"] == "00024") & (int_res["평형타입"] == 'C')][["아파트코드","평형","평형타입","조사일", "실거래가", "실거래_중복_num","114_보정후_하한가", "114_보정후_상한가"]].sort_values(by=["조사일"])

In [ ]:
int_res.head()

In [ ]:
# from datetime import date
dt = datetime.datetime.now().isoformat()[:10].replace("-","")
int_res = int_res.reset_index(drop=True)
int_res.to_csv("과거시세보정(매매)"+ "_" + dt + "_41_time(전국)" +  ".csv", index=False, encoding="CP949")
print(time.time() - start)